In [1]:
import sys
sys.path.append('/work/pikachu/utils/')
sys.path.append('/work/pikachu/third/')
sys.path.append('..')
from gezi.common import *
import gezi
from src import config
from src.config import *
from src import util
from src.dataset import Dataset
from src.eval import eval_seq
gezi.init_flags()

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/home/gezi/miniconda3/lib/python3.10/site-packages/wandb/integration/keras/keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/home/gezi/miniconda3/lib/python3.10/site-packages/wandb/integration/keras/keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/home/gezi/miniconda3/lib/python3.10/site-packages/wandb/integration/keras/keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
[

In [2]:
V = 29
fold = 0
model_names = [
    'conformer-fat.encoder_layers-15.time_reduce.time_reduce_idx-7.relpos_combine_mode-2.fixreduce.eval',
    'conformer-fat.encoder_layers-15.encoder-conformer_v5_1_11.encode_pool_size-1.reduce_idxes-8,13.eval',
  ]
root = f'../working/offline/{V}/{fold}'
model_paths = [f'{root}/{model_name}' for model_name in model_names]
gezi.restore_configs(model_paths[0])

In [3]:
FLAGS.mn = 'ensemble'
config.init()
mt.init()
config.show()

[08/18/23 19:24:45] config.py:514 in init()
                    len(FLAGS.train_files): 212
                    len(FLAGS.valid_files): 0
[08/18/23 19:24:45] config.py:567 in init()
                    FLAGS.n_frames: 320
                    FLAGS.encode_pool_size: 2
                    FLAGS.encode_out_feats: None
[08/18/23 19:24:45] config.py:580 in init()
                    FLAGS.n_frames: 320
                    FLAGS.encode_pool_size: 2
                    FLAGS.encode_out_feats: 160
[08/18/23 19:24:45] config.py:598 in init()
                    FLAGS.inter_models: [15, 90.0]


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[08/18/23 19:24:45] init.py:400 in init()
                    FLAGS.model_dir: '../working/offline/29/0/ensemble.fixreduce'
[08/18/23 19:24:45] init.py:552 in init()
                    tf.__version__: '2.13.0'
                    torch.__version__: '2.0.1+cu118'
[08/18/23 19:24:46] init.py:1325 in init()
                    global_batch_size: 128
                    acc_steps: 1
                    replica_batch_size: 128
                    eval_batch_size: 128
                    replica_eval_batch_size: 128
                    num_gpus: 1
                    gpus: [2]
                    CUDA_VISIABLE_DEVICES: []
                    num_threads: 32
                    FLAGS.num_workers: 4
                    work_mode: 'valid'
                    distributed: False
                    seed: 2352
[08/18/23 19:24:46] init.py:1580 in init()
                    FLAGS.model_name: 'ensemble.fixreduce'
                    FLAGS.model_dir: '../working/offline/29/0/ensemble.fixreduce'
[08/1

In [4]:
record_dir = f'{FLAGS.root}/tfrecords/0.1'
records_pattern = f'{record_dir}/*.tfrec'
files = gezi.list_files(records_pattern) 
FLAGS.valid_files = [x for x in files if int(os.path.basename(x).split('.')[0]) % FLAGS.folds == FLAGS.fold]
dataset = Dataset('valid', files=FLAGS.valid_files)
# means = gezi.load(f'{record_dir}/means.npy')
# stds = gezi.load(f'{record_dir}/stds.npy')
# STATS['means'] = means
# STATS['stds'] = stds
datas = dataset.make_batch(256)

[08/18/23 19:24:58] preprocess.py:99 in __init__()- self.n_cols: 769
[08/18/23 19:24:58] preprocess.py:102 in __init__()
                    self.means.shape: (384,)
                    self.stds.shape: (384,)
[08/18/23 19:24:59] dataset.py:502 in make_batch()
                    self.subset: 'valid'
                    repeat: False
                    drop_remainder: False


In [5]:
def get_model(model_path):
  model_path_ = model_path.replace('.eval', '')
  ic(model_path_)
  gezi.restore_configs(model_path_)
  ic(FLAGS.encoder_layers, FLAGS.encoder_units, FLAGS.n_frames)
  model = util.get_model()
  model = util.prepare_tflite(model)
  model(iter(datas).next()[0]['frames'])
  model.load_weights(f'{model_path}/tflite.h5')
  # model.get_model().summary()
  ic(pd.read_csv(f'{model_path}/metrics.csv').tail(1)['score'].values[0])
  return model

In [7]:
models = [get_model(model_path) for model_path in tqdm(model_paths)]
models

  0%|          | 0/2 [00:00<?, ?it/s]

[08/18/23 19:28:35] 3239111075.py:3 in get_model()
                    model_path_: '../working/offline/29/0/conformer-fat.encoder_layers-15.time_reduce.time_reduce_idx-7.relpos_combine_mode-2.fixreduce'
[08/18/23 19:28:35] 3239111075.py:5 in get_model()
                    FLAGS.encoder_layers: 15
                    FLAGS.encoder_units: 192
                    FLAGS.n_frames: 320


[08/18/23 19:28:35] util.py:84 in torch2keras()
                    input_shape: [1, 320, 769]


Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

 (!) Max diff 0.000708  
InferModel[src.torch.models.encoder](float32_0<1,320,769>) -> float32_2648<1,80,61>
 │   (!) Max diff 0.001298  
 │  Encoder[src.torch.encoder](float32_0<1,320,769>) -> float32_2645<1,80,192>
 │   │   (!) Max diff 0.001640  
 │   │  Encoder[src.torch.encoders.conformer_v5_1_8](float32_0<1,320,769>) -> float32_2642<1,160,192>
 │   │   │   (!) Max diff 0.000738  
 │   │   │  SimpleEmbedding[src.torch.embedding](float32_0<1,320,769>) -> float32_9<1,320,192>
 │   │   │   │   (!) Max diff 0.000738  
 │   │   │   │  Linear[torch.nn.modules.linear](float32_0<1,320,769>) -> float32_2<1,320,192>
 │   │   │   │   └· l

KeyError: 16923581157668698623

In [10]:
model = mt.EnsembleModel(models, activation='softmax')
model(iter(datas).next()[0])
model.get_model().summary()

Layer model_0_0 is not connected, no input to return.
Model: "ensemble_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_0_0 (Model)           multiple                  4627352   
                                                                 
 model_2_1_1 (Model)         multiple                  6797705   
                                                                 
Total params: 11425058 (43.58 MB)
Trainable params: 11385250 (43.43 MB)
Non-trainable params: 39808 (155.50 KB)
_________________________________________________________________


In [11]:
num_steps = dataset.num_steps
eval_seq(datas, model, num_steps)

eval_loop:   0%|          | 0/66 [00:00<?, ?it/s]

[07/23/23 08:19:35] eval.py:161 in eval_seq()
                    df.head(20):       sequence_id phrase_type  phrase_dup                     phrase_true  \
                                 3688   1816825349         url           0                 scales/kuhaylah   
                                 3556   1816909464     address           1             1383 william lanier   
                                 3667   1817169529         url           1          https://jsi.is/hukuoka   
                                 3527   1817348598     address           1                  wildberries_ru   
                                 3525   1817606833     address           1            6867 granville drive   
                                 3530   1817770333     address           1                 166 water power   
                                 3700   1818125771         url           1      www.theorienttreasures.com   
                                 3677   1818160881     address           1

{'phrase_len_true': 17.74735150577312,
 'phrase_len_pred': 15.186287346744436,
 'char/max_idx_max': 79,
 'char/max_idx': 40.98208546601595,
 'char/ori_rate': 0.5122760683251995,
 'char/true_rate': 2.3426042690290023,
 'char/pred_rate': 2.8874561082813526,
 'distance': 4.198131174860135,
 'acc/char': 0.9556045587793222,
 'acc/type': 0.9491132008094274,
 'acc/first': 0.7848470420188073,
 'acc/last': 0.7249137007499107,
 'score': 0.7634502718056547,
 'phrase_len_true/phone': 14.309203980099502,
 'phrase_len_pred/phone': 12.553233830845771,
 'char/max_idx_max/phone': 79,
 'char/max_idx/phone': 38.76417910447761,
 'char/ori_rate/phone': 0.48455223880597015,
 'char/true_rate/phone': 2.7120313664092057,
 'char/pred_rate/phone': 3.189342930329192,
 'distance/phone': 2.8358208955223883,
 'acc/char/phone': 0.9757821064170672,
 'acc/type/phone': 0.9507462686567164,
 'acc/first/phone': 0.8201492537313433,
 'acc/last/phone': 0.7430348258706467,
 'score/phone': 0.8018184030735532,
 'phrase_len_true/

: 

In [9]:
gezi.get('eval_df')

,sequence_id,phrase_type,phrase_dup,phrase_true,phrase_pred,phrase_ori,char/max_idx,char/ori_rate,char/true_rate,char/pred_rate,n_frame,phrase_len_true,phrase_len_pred,idx,distance,acc/char,acc/type,acc/first,acc/last,score
3688,1816825349,url,0,scales/kuhaylah,scales/kuhaylan,<PAD><PAD>ss<PAD>ccalleess<PAD>//<PAD><PAD><PA...,37,0.4625,2.466667,2.466667,127,15,15,0,1,0.983051,True,True,False,0.933333
3556,1816909464,address,1,1383 william lanier,1383 william lanier,<PAD><PAD><PAD>11<PAD><PAD><PAD>33<PAD><PAD>8<...,61,0.7625,3.210526,3.210526,236,19,19,1,0,1.000000,True,True,True,1.000000
3667,1817169529,url,1,https://jsi.is/hukuoka,https://jsi.is/htkuoka,<PAD><PAD>hht<PAD><PAD>tttpppss<PAD>:::<PAD>//...,75,0.9375,3.409091,3.409091,300,22,22,2,1,1.000000,True,True,True,0.954545
3527,1817348598,address,1,wildberries_ru,wild eres_,<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><...,33,0.4125,2.357143,3.300000,120,14,10,3,5,0.949153,True,True,False,0.642857
3525,1817606833,address,1,6867 granville drive,6867 hranvile drive,<PAD><PAD><PAD><PAD><PAD><PAD>6<PAD>88<PAD><PA...,49,0.6125,2.450000,2.578947,182,20,19,4,2,0.966102,True,True,True,0.900000
3530,1817770333,address,1,166 water power,2w teowil,<PAD><PAD>2<PAD><PAD><PAD><PAD><PAD><PAD>w <P...,32,0.4000,2.133333,3.555556,117,15,9,5,10,0.864407,True,False,False,0.333333
3700,1818125771,url,1,www.theorienttreasures.com,www.theorienttreasures.com,<PAD><PAD><PAD><PAD>ww<PAD>w<PAD><PAD>wwwwww.....,79,0.9875,3.038462,3.038462,396,26,26,6,0,1.000000,True,True,True,1.000000
3677,1818160881,address,1,9345 happy dirt,9345 papy bert,<PAD><PAD><PAD><PAD>9<PAD><PAD>3344<PAD><PAD>5...,36,0.4500,2.400000,2.571429,130,15,14,7,4,0.915254,True,True,True,0.733333
3539,1818216000,address,1,5126 whitewind drive,5126 whiterwind drive,<PAD>5<PAD><PAD>122<PAD>66 whhiitteerwii<PAD>...,40,0.5000,2.000000,1.904762,151,20,21,8,1,1.000000,True,True,True,0.950000
3591,1818246565,phone,0,893-125-1663,893-125-163,<PAD>88<PAD>99<PAD>3--1<PAD><PAD>2<PAD>55-11<P...,26,0.3250,2.166667,2.363636,92,12,11,9,1,1.000000,True,True,True,0.916667
